In [16]:
import requests
import json
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime, timezone
import time

In [17]:
from opensky_api import OpenSkyApi
from config import opensky_user, opensky_pass

```
$ curl -u "USERNAME:PASSWORD" -s "https://opensky-network.org/api/flights/arrival?airport=EDDF&begin=1517227200&end=1517230800" | python -m json.tool
```

In [18]:
def datetime_to_unix(dt):
    return int(time.mktime(dt.timetuple()))

## POC - retrieve state vectors for a certain area
https://openskynetwork.github.io/opensky-api/python.html#examples

In [19]:
api = OpenSkyApi(opensky_user, opensky_pass)

states = api.get_states(bbox=(45.8389, 47.8229, 5.9962, 10.5226))
for s in states.states:
    print("(%r, %r, %r, %r)" % (s.longitude, s.latitude, s.baro_altitude, s.velocity))

(8.5107, 47.5783, 11582.4, 213.44)
(8.5589, 47.457, None, 0)
(8.5495, 47.4722, None, 0.45)
(8.5589, 47.4569, None, 0)
(9.0944, 47.2021, 10972.8, 212.62)
(9.3651, 47.1676, 11277.6, 263.25)
(8.1071, 47.7579, 10972.8, 213.56)
(10.0919, 46.6928, 11582.4, 209.37)
(7.7078, 47.725, 1165.86, 67.52)
(9.0727, 47.252, 12192, 209.05)
(8.5637, 47.4421, 365.76, 0)


## POC - retrieve arrivals and departures for a specific airport and time interval
https://openskynetwork.github.io/opensky-api/python.html#examples

In [20]:
airport = 'KMSP'

start_time = datetime(2024, 7, 15, 0, 0)
end_time = datetime(2024, 7, 15, 23, 59)

start_ts = datetime_to_unix(start_time)
end_ts = datetime_to_unix(end_time)

arrivals = api.get_arrivals_by_airport(airport, start_ts, end_ts)
departures = api.get_departures_by_airport(airport, start_ts, end_ts)

print("Arrivals:")
for flight in arrivals:
    print(flight)

print("Departures:")
for flight in departures:
    print(flight)

Arrivals:
{   'arrivalAirportCandidatesCount': 251,
    'callsign': 'FFT1292 ',
    'departureAirportCandidatesCount': 171,
    'estArrivalAirport': 'KMSP',
    'estArrivalAirportHorizDistance': 1601,
    'estArrivalAirportVertDistance': 40,
    'estDepartureAirport': 'KDEN',
    'estDepartureAirportHorizDistance': 5066,
    'estDepartureAirportVertDistance': 97,
    'firstSeen': 1721093128,
    'icao24': 'a98f2d',
    'lastSeen': 1721098885}
{   'arrivalAirportCandidatesCount': 12,
    'callsign': 'DAL2483 ',
    'departureAirportCandidatesCount': 0,
    'estArrivalAirport': 'KMSP',
    'estArrivalAirportHorizDistance': 10782,
    'estArrivalAirportVertDistance': 536,
    'estDepartureAirport': 'CYVR',
    'estDepartureAirportHorizDistance': 748,
    'estDepartureAirportVertDistance': 42,
    'firstSeen': 1721074454,
    'icao24': 'acd4d7',
    'lastSeen': 1721084392}
{   'arrivalAirportCandidatesCount': 10,
    'callsign': 'N850TP  ',
    'departureAirportCandidatesCount': 0,
    'es